# Tools: Code Intepreter

In [1]:
from openai import OpenAI
import json
from dotenv import load_dotenv
import time

In [2]:
load_dotenv()

True

In [3]:
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

In [4]:
client = OpenAI()

**Define the instructions and user question**

Instructions:
- __You are a Python developer that is working on creating a travel app. When asked a question, write and run code to answer the question.__

User question:
- __Calculate the tip for a bill of $175. The restaurant requests 20% and we want to divide the total bill with the tip into 4 equal parts.__

In [5]:
instructions = "You are a Python developer that is working on creating a travel app. When asked a question, write and run code to answer the question."

In [6]:
user_question = "Calculate the tip for a bill of $175. The restaurant requests 20% and we want to divide the total bill with the tip into 4 equal parts."

**Create Assistant enabling Code Interpreter, create Thread and Run, and execute**

In [7]:
assistant = client.beta.assistants.create(
  instructions=instructions,
  model="gpt-4-1106-preview",
  tools=[{"type": "code_interpreter"}]
)

In [8]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": user_question,
    }
  ]
)

In [9]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

In [10]:
while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)

print("Run completed")

Run completed


**Get messages in desc order, print out last assistant response**

In [11]:
messages = client.beta.threads.messages.list(thread_id=thread.id, order="desc")

**The Assistant created the code internally and executed in a sandbox**

In [12]:
messages.data[0].content[0].text.value

'The tip for the bill of $175 at 20% is $35. The total bill with the tip is $210. When divided into 4 equal parts, each person would pay $52.50.'

## An example that uses a data file as source

**Upload the file that you would like the assistant to use**

In [13]:
travel_meals = "travelmeals.csv"

In [14]:
file = client.files.create(
    file=open(
        travel_meals,
        "rb",
    ),
    purpose="assistants",
)

In [16]:
print(file)

FileObject(id='file-YX699wYxMafpZtjK744Xm1PY', bytes=493, created_at=1703115261, filename='travelmeals.csv', object='file', purpose='assistants', status='processed', status_details=None)


**Update the assistant to include a file**

_Quick note: Files can also be specified at thread level._

In [17]:
assistant = client.beta.assistants.update(
    assistant.id,
    tools=[{"type": "code_interpreter"}],
    file_ids=[file.id],
)

In [18]:
show_json(assistant)

{'id': 'asst_rK8ksDvTI1UhLvFKJSfK6wzb',
 'created_at': 1703115223,
 'description': None,
 'file_ids': ['file-YX699wYxMafpZtjK744Xm1PY'],
 'instructions': 'You are a Python developer that is working on creating a travel app. When asked a question, write and run code to answer the question.',
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'name': None,
 'object': 'assistant',
 'tools': [{'type': 'code_interpreter'}]}

**A few functions to execute Runs (same code as used in previous demos)**

In [19]:
def add_message(the_assistant_id, the_thread_id, user_message):
    client.beta.threads.messages.create(
        thread_id=the_thread_id, role="user", content=user_message
    )
    return client.beta.threads.runs.create(
        thread_id=the_thread_id,
        assistant_id=the_assistant_id,
    )

In [20]:
def get_messages(the_thread_id):
    return client.beta.threads.messages.list(thread_id=the_thread_id, order="desc")

In [21]:
def wait_for_run_completion(run, thread_id):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread_id,
            run_id=run.id,
        )
        time.sleep(0.1)
    return run

In [22]:
def ask_question(user_message, assistant_id, thread_id):   
    run = add_message(assistant_id, thread_id, user_message)
    wait_for_run_completion(run, thread_id)   
    the_response = get_messages(thread_id)
    print("Assistant:\n" + the_response.data[0].content[0].text.value)

def ask_question(user_message, assistant_id, thread_id):   
    run = add_message(assistant_id, thread_id, user_message)
    wait_for_run_completion(run, thread_id)   
    the_response = get_messages(thread_id)
    print("Assistant:\n" + the_response.data[0].content[0].text.value)

**Ask for a piece of code that can run and group rows by total**

In [23]:
user_message = "The file travelmeals.csv contains a list of meals for the last trip of the company CEO. Write a piece of code that groups entries by the column type and create an output CSV with the totals. The first row corresponds to the header."

In [24]:
ask_question(user_message, assistant.id, thread.id)

Assistant:
I have grouped the entries by `mealtype` and calculated the totals for the `total` and `tip` columns. The results have been saved to a new CSV file named `totals_by_mealtype.csv`. Here are the totals for each meal type:

- Breakfast: $50.25 total, $5.00 tip
- Dinner: $448.30 total, $44.50 tip
- Lunch: $125.30 total, $12.50 tip

You can download the output CSV file using the following link: [totals_by_mealtype.csv](sandbox:/mnt/data/totals_by_mealtype.csv).


In [25]:
csv_messages = get_messages(thread.id)

show_json(csv_messages)

{'data': [{'id': 'msg_p3qOPC27R6ufPgLoxqrNtp7O',
   'assistant_id': 'asst_rK8ksDvTI1UhLvFKJSfK6wzb',
   'content': [{'text': {'annotations': [{'end_index': 458,
        'file_path': {'file_id': 'file-P4U67eM63GrguL84xrJ1i0dr'},
        'start_index': 418,
        'text': 'sandbox:/mnt/data/totals_by_mealtype.csv',
        'type': 'file_path'}],
      'value': 'I have grouped the entries by `mealtype` and calculated the totals for the `total` and `tip` columns. The results have been saved to a new CSV file named `totals_by_mealtype.csv`. Here are the totals for each meal type:\n\n- Breakfast: $50.25 total, $5.00 tip\n- Dinner: $448.30 total, $44.50 tip\n- Lunch: $125.30 total, $12.50 tip\n\nYou can download the output CSV file using the following link: [totals_by_mealtype.csv](sandbox:/mnt/data/totals_by_mealtype.csv).'},
     'type': 'text'}],
   'created_at': 1703115359,
   'file_ids': ['file-P4U67eM63GrguL84xrJ1i0dr'],
   'metadata': {},
   'object': 'thread.message',
   'role': 'ass

In [26]:
csv_file = csv_messages.data[0].content[0].text.annotations[0].file_path.file_id

print(csv_file)

file-P4U67eM63GrguL84xrJ1i0dr


In [27]:
file_data = client.files.content(csv_file)

In [28]:
file_bytes = file_data.read()

In [29]:
with open("./result.csv", "wb") as file:
    file.write(file_bytes)

**Now, let's do a graph**

In [37]:
# A good test is to show either with matplotlib or seaborn
user_message = "The file travelmeals.csv contains a list of meals for the last trip of the company CEO. Write a piece of code that groups entries by the column type and create as output a pie chart that shows entries grouped by meal type. Use seaborn as graph library. The first row corresponds to the header."

In [38]:
run = add_message(assistant.id, thread.id, user_message)
wait_for_run_completion(run, thread.id) 

Run(id='run_Wklpv1WLGbe8qrAKjs7QPzev', assistant_id='asst_rK8ksDvTI1UhLvFKJSfK6wzb', cancelled_at=None, completed_at=1703115496, created_at=1703115483, expires_at=None, failed_at=None, file_ids=['file-YX699wYxMafpZtjK744Xm1PY'], instructions='You are a Python developer that is working on creating a travel app. When asked a question, write and run code to answer the question.', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=1703115483, status='completed', thread_id='thread_bUkh4zNmz2Bwm2wMMk8idMuu', tools=[ToolAssistantToolsCode(type='code_interpreter')])

In [39]:
image_messages = get_messages(thread.id)

In [40]:
print(image_messages)

SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_iYBG4GFdDMyNtF4m3e5nAxQ1', assistant_id='asst_rK8ksDvTI1UhLvFKJSfK6wzb', content=[MessageContentImageFile(image_file=ImageFile(file_id='file-29gEEgQrA7XYej5LE36m98XL'), type='image_file'), MessageContentText(text=Text(annotations=[], value='Here is the pie chart, created using matplotlib\'s pie function with colors from Seaborn\'s "pastel" color palette, that shows the entries grouped by meal type.'), type='text')], created_at=1703115495, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_Wklpv1WLGbe8qrAKjs7QPzev', thread_id='thread_bUkh4zNmz2Bwm2wMMk8idMuu'), ThreadMessage(id='msg_R5hzmcqoMcc2yp1UdYBIgvhP', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='The file travelmeals.csv contains a list of meals for the last trip of the company CEO. Write a piece of code that groups entries by the column type and create as output a pie chart that shows entries grouped by meal

In [41]:
image_file_id = image_messages.data[0].content[0].image_file.file_id

print(image_file_id)

file-29gEEgQrA7XYej5LE36m98XL


In [42]:
raw_response = client.files.with_raw_response.content(file_id=image_file_id)

In [43]:
with open("chart-seaborn.png", "wb") as f:
    f.write(raw_response.content)